In [ ]:
import numpy as np
from openpiv import tools, pyprocess, scaling, validation
from openpiv.deep_refinement import DeepRefiner # The new module

# 1. Load Images
frame_a = tools.imread( 'exp1_001_a.bmp' )
frame_b = tools.imread( 'exp1_001_b.bmp' )

# 2. Standard OpenPIV (The "Coarse" Step)
# We use standard WIDIM or simple Cross-Correlation
u, v, sig2noise = pyprocess.extended_search_area_piv(
    frame_a.astype(np.int32), 
    frame_b.astype(np.int32), 
    window_size=24, 
    overlap=12, 
    dt=0.02, 
    search_area_size=64, 
    sig2noise_method='peak2peak'
)

# Get the grid coordinates for the coarse vectors
x, y = pyprocess.get_coordinates(
    image_size=frame_a.shape, 
    search_area_size=64, 
    overlap=12
)

# 3. Filter Outliers (Important before feeding to CNN)
u, v, mask = validation.sig2noise_val( u, v, sig2noise, threshold = 1.3 )
u, v = validation.global_val( u, v, (-1000, 1000), (-1000, 1000) )
u, v = tools.replace_outliers( u, v, method='localmean', max_iter=10, kernel_size=2)

# 4. Apply CNN Refinement (The Choi et al. Step)
# Initialize refiner (optionally load weights for RAFT/FlowNet2)
refiner = DeepRefiner(model_path="weights/raft-sintel.pth")

# Get pixel-dense, super-resolved flow
u_dense, v_dense = refiner.refine(frame_a, frame_b, u, v, x, y)

# 5. Save/Plot
tools.save(x, y, u, v, mask, 'openpiv_result.txt')
# Save the dense field (custom function needed as it's pixel-wise, not grid-wise)
np.savez('dense_result.npz', u=u_dense, v=v_dense)